# NYC Delivery Promise Engine: Business Strategy & Policy Analysis

## Notebook Overview
This notebook analyzes delivery promise strategies and their business trade-offs:
1. **P50 vs P90 Strategy Analysis**: Customer satisfaction vs operational reliability
2. **Borough & Time-Based Insights**: Geographic and temporal optimization opportunities  
3. **Weather Impact Assessment**: Dynamic promise adjustments for conditions
4. **Business Recommendations**: Actionable insights for delivery operations

### Key Business Questions:
- Should we promise P50 (faster, more late deliveries) or P90 (slower, fewer late deliveries)?
- How do location and time affect delivery performance?
- What dynamic adjustments improve customer satisfaction?


In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
from datetime import datetime

print("🎯 NYC Delivery Promise Engine - Business Strategy Analysis")
print("=" * 70)

# Load test data for analysis
df = pd.read_csv('../data/processed_test_data.csv')
print(f"✅ Analysis dataset: {len(df):,} July 2025 trips")

# Calculate key percentiles for promise strategy
p50 = df['trip_duration_minutes'].quantile(0.5)
p90 = df['trip_duration_minutes'].quantile(0.9)
p95 = df['trip_duration_minutes'].quantile(0.95)

print(f"\n📊 Delivery Promise Benchmarks:")
print(f"P50 (Median): {p50:.1f} minutes")
print(f"P90: {p90:.1f} minutes")
print(f"P95: {p95:.1f} minutes")


In [ ]:
# Promise Strategy Analysis: P50 vs P90
print("\n⚖️ Promise Strategy Trade-off Analysis")
print("-" * 50)

# Calculate late delivery rates for different promise strategies
p50_late_rate = (df['trip_duration_minutes'] > p50).mean()
p90_late_rate = (df['trip_duration_minutes'] > p90).mean()

print(f"📋 Promise Strategy Comparison:")
print(f"P50 Strategy: {p50:.1f} min promise, {p50_late_rate:.1%} late deliveries")
print(f"P90 Strategy: {p90:.1f} min promise, {p90_late_rate:.1%} late deliveries")

# Buffer time analysis
p50_buffer = 0  # By definition
p90_buffer = p90 - p50

print(f"\n⏱️ Customer Wait Time Impact:")
print(f"P50 Strategy: {p50_buffer:.1f} min buffer")
print(f"P90 Strategy: +{p90_buffer:.1f} min longer wait")

# Borough-specific analysis
borough_stats = df.groupby('pickup_borough')['trip_duration_minutes'].agg([
    'count', 'mean', 'median', 
    lambda x: x.quantile(0.9)
]).round(1)
borough_stats.columns = ['trips', 'avg_duration', 'median_duration', 'p90_duration']
borough_stats = borough_stats.sort_values('avg_duration', ascending=False)

print(f"\n🏙️ Borough Performance Rankings:")
print(borough_stats)

# Generate business recommendations
print(f"\n🎯 Key Business Insights:")
print(f"1. P90 promises reduce late deliveries by {(p50_late_rate - p90_late_rate)*100:.0f} percentage points")
print(f"2. Trade-off: +{p90_buffer:.1f} min average wait time for reliability")
print(f"3. Borough variation: {borough_stats['avg_duration'].max():.1f} to {borough_stats['avg_duration'].min():.1f} min")

# Export key metrics for dashboard
metrics = {
    'analysis_date': datetime.now().isoformat(),
    'dataset_size': len(df),
    'p50_minutes': float(p50),
    'p90_minutes': float(p90),
    'p50_late_rate': float(p50_late_rate),
    'p90_late_rate': float(p90_late_rate),
    'promise_buffer_minutes': float(p90_buffer),
    'borough_performance': borough_stats.to_dict('index')
}

# Save metrics
with open('../artifacts/metrics.json', 'w') as f:
    json.dump(metrics, f, indent=2)
    
print(f"\n💾 Business metrics exported to artifacts/metrics.json")
print(f"📊 Ready for executive dashboard and deployment!")
